In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import json

In [25]:
tickers = ["VT", "^GSPC", "0050.TW", "CIBR"]
df = yf.download(tickers, period="5d")["Low"].dropna().tail(1)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  4 of 4 completed


In [ ]:
def send_line_message(msg):
    try:
        import os
        from linebot import LineBotApi
        from linebot.models import TextSendMessage
        line_bot_api = LineBotApi(os.getenv('CHANNEL_ACCESS_TOKEN'))
        message = TextSendMessage(text=msg)
        line_bot_api.push_message(os.getenv('USER_ID'), message)
    except Exception:
        raise

In [ ]:
try:
    with open('Records.json', 'r+', encoding='utf-8') as f:
        recs = json.load(f)
        data_changed = False
        
        for c in df.columns:
            price = df[c].iloc[-1]
            stock = recs[c]
            drawdown = (price - stock['High']) / stock['High']

            if price >= stock['High']:
                from datetime import datetime
                now = datetime.now()
                stock['Date'] = df[c].index.item().strftime("%Y-%m-%d")
                stock['High'] = price
                stock['Threshold'] = -0.05
                stock['Notified'] = False
                data_changed = True
                continue

            elif drawdown <= stock['Threshold']:
                send_line_message(f'昨日股價自前高點回撤{drawdown*100:.2f}%，準備加碼買進！')
                stock['Notified'] = True
                stock['Threshold'] -= 0.05
                data_changed = True
            
        # 寫入json檔
        if data_changed:
            f.seek(0)
            json.dump(recs, f, ensure_ascii=False, indent=4)
            f.truncate()
except Exception:
    send_line_message(f'Github Actions 出現錯誤')

In [3]:
df = yf.download('^GSPC', period="5d", auto_adjust=True)["Low"].tail(1)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^GSPC']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [4]:
tickers = ["VT", "^GSPC", "0050.TW", "CIBR"]
df = yf.download(tickers, period="5d", auto_adjust=True)

[*********************100%***********************]  4 of 4 completed

4 Failed downloads:
['0050.TW', 'CIBR', '^GSPC', 'VT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
